In [20]:
from keras.layers import Input, Conv3D, MaxPooling3D, Dense, Flatten, ZeroPadding3D, Dropout
from keras.models import Model
import keras.backend as K
import keras.losses
keras.losses.custom_loss = custom_loss

def ODAS():
    """Instantiates a ODAS Keras model
    
    Returns:
    A Keras model.
    
    """
    
    if K.image_data_format() == 'channels_last':
        shape0 = (16,112,112,3)
    else:
        shape0 = (3,16,112,112)
        
        
    main_input = Input(shape=shape0, dtype='float32', name='main_input')
    
    conv1 = Conv3D(64, 3, activation='relu', padding='same', name='conv1')(main_input)
    pool1 = MaxPooling3D(pool_size=(1,2,2), strides=(1,2,2), padding='same', name='pool1')(conv1)
    
    conv2 = Conv3D(128, 3, activation='relu', padding='same', name='conv2')(pool1)
    pool2 = MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding='valid', name='pool2')(conv2)
    
    conv3a = Conv3D(256, 3, activation='relu', padding='same', name='conv3a')(pool2)
    conv3b = Conv3D(256, 3, activation='relu', padding='same', name='conv3b')(conv3a)
    pool3 = MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding='valid', name='pool3')(conv3b)
    
    conv4a = Conv3D(512, 3, activation='relu', padding='same', name='conv4a')(pool3)
    conv4b = Conv3D(512, 3, activation='relu', padding='same', name='conv4b')(conv4a)
    pool4 = MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding='valid', name='pool4')(conv4b)
    
    conv5a = Conv3D(512, 3, activation='relu', padding='same', name='conv5a')(pool4)
    conv5b = Conv3D(512, 3, activation='relu', padding='same', name='conv5b')(conv5a)
    zero = ZeroPadding3D(padding=(0,1,1))(conv5b)
    pool5 = MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding='valid', name='pool5')(zero)
    
    flat = Flatten()(pool5)
    
    fc6= Dense(4096, activation='relu', name='fc6')(flat)
    drop1 = Dropout(0.5)(fc6)
    fc7 = Dense(4096, activation='relu', name='fc7')(drop1)
           
    model_fc7 = Model(main_input, fc7)
   
    start_window = Input(shape=shape0)
    followup_window = Input(shape=shape0)
    
    out1 = model_fc7(start_window)
    out2 = model_fc7(followup_window)
    
    drop2 = Dropout(0.5)(out1)
    fc8 = Dense(2, activation='softmax', name='fc8')(drop2)
    
    model = Model([start_window,followup_window],fc8)
    
    model.compile(optimizer='Adam', loss='custom_loss')
    
    return model

def loss_classification(y_true, y_pred):
    return -K.mean(K.log(K.dot(y_pred,K.transpose(y_true))))
    
def loss_temporal_consistency(out1, out2):
    return K.mean(K.square(out1 - out2), axis=-1)
    
def custom_loss(y_true, y_pred):
    return loss_classification(y_true, y_pred) + loss_temporal_consistency(out1, out2)

ODAS()

NameError: ignored